# Using updated astrometry solutions

As part of an effort to improve the absolute astrometry of HST images, STScI has created multiple new astrometric solutions for ACS and WFC3 images.  These solutions are contained in the World Coordinate System (WCS) of the images, as well as headerlet extesions.  This notebook provides an example workflow showing how to use/change to a different solution.

The solutions are contained in the exposure files (`flt.fits` and `flc.fits`), so no extra files are required.  However, these updates were implemented in December 2019 as a part of HST Data Processing version 2019.5.  If data were downloaded before that, they can either be redownloaded (and will contain the new solutions) from mast.stsci.edu or via astroquery, or can be updated by connecting to the database cotaining solutions (shown below).

For more information, see the overview page here: https://outerspace.stsci.edu/pages/viewpage.action?spaceKey=HAdP&title=Improvements+in+HST+Astrometry


<div class="alert alert-block alert-info">
<b>REQUIREMENT:</b> The software included in this notebook were ran using stwcs version 1.5.3, in python 3.6.  If versions older than this are used <b>the software will likely not work</b>!
</div>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> The new solutions are used by default by the MAST pipeline!  Some datasets may have certain solutions  such as fitting to Gaia DR2, though other datasets (even in the same visit) may not!  Thus, it is crucial to check the which solution (WCS) is active!  The easiest way to check which solution is active is to check the WCSNAME keyword in the header of the SCI extensions.
</div>

# TOC

 0. [Example Data Download](#0.-Example-Data-Download)
 1.  [New Extensions on FITS Files](#1.-New-extensions-on-fits-files)
 2. [Exploring different solutions](2.-Exploring-different-solutions)
 3. [Applying a headerlet to the science extensions](#3.-Applying-a-headerlet-to-the-science-extensions)
 4. [Restoring the "old" solution](#4.-Restoring-the-"old"-solution)
 5. [Inspecting alignment](#5.-Inspecting-alignment-(optional))
 6. [Changing the WCS of drizzled images](#6.-Changing-the-WCS-of-drizzled-images)

In [1]:
import glob
import numpy as np
import os
import shutil

from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Observations
from stwcs.wcsutil import headerlet
from stwcs import updatewcs

## 0. Example Data Download

In [ ]:
obsTable = Observations.query_criteria(project='HST',proposal_id='14689',obs_id='ID7307030')
products = Observations.get_product_list(obsTable)
filtered_products = Observations.filter_products(products,mrp_only=False,productSubGroupDescription=['FLC', 'ASN'])
filtered_products

In [ ]:
dl_tbl = Observations.download_products(filtered_products,mrp_only=False)
dl_tbl

In [ ]:
for f in dl_tbl['Local Path']:
    filename = os.path.split(f)[-1]
    if not os.path.exists(filename):
        shutil.move(f, '.')

## 1. New extensions on fits files

Using `fits.info` prints basic information about the extensions in a fits file.  In the following examples, we will show operations for one file, though performing the same operations for multiple files simply requires looping.

In [2]:
filename = 'id7307xfq_flc.fits'

If no extensions named `HDRLET` appear, then solutions can be downloaded from the database (if available) and automatically appended to the fits files using the following command:

In [ ]:
updatewcs.updatewcs(filename, use_db=True)

The updated solutions should then show up as extra `HDRLET` extensions

In [ ]:
fits.info(filename)

As seen above, there are new `HDRLET` extensions in the fits files (compared to the pre-2019.3 products).  These extensions each contain information used to construct a World Coordinate System (WCS), which is used to transform image coordinates into physical (sky) coordinates.  Each of these WCS's represent an astrometric solution derived in a different way.

## 2. Exploring different solutions

Each HDRLET extension contains information describing the solution used in its creation. To investigate this we first programmatically obtain the extension numbers of the HDRLETs.

In [ ]:
ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)

# To show it's consistent with the fits.info from above
print(ext_indices)

We can then loop through these extensions to see what the WCS solutions are.

In [ ]:
hdu = fits.open(filename)
print('Ext\tWCSNAME')
for ext_ind in ext_indices:
    print(ext_ind, '\t', hdu[ext_ind].header['WCSNAME'])

hdu.close()

Alternatively, we can use `get_headerlet_kw_names`:

In [ ]:
new_wcsnames = headerlet.get_headerlet_kw_names(filename, kw='WCSNAME')

In [ ]:
new_wcsnames

We can write this into a convenience function:

In [ ]:
def get_hdrlet_wcsnames(filename):
    """Print and return list of WCS names in HDRLET extensions of fits file"""
    hdu = fits.open(filename)
    ext_indices = headerlet.find_headerlet_HDUs(filename, strict=False)

    print('Ext\tWCSNAME')
    new_wcsnames = []
    for ext_ind in ext_indices:
        name = hdu[ext_ind].header['WCSNAME']
        print(ext_ind, '\t', name)
        new_wcsnames.append(name)
        
    hdu.close()
    return new_wcsnames

In [ ]:
new_wcsnames = get_hdrlet_wcsnames(filename)

We can also see which solution is the "active" solution (the one currently applied to the SCI extensions):

In [ ]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

The nature of each solution is described here: https://drizzlepac.readthedocs.io/en/latest/astrometry.html#interpreting-wcs-names

## 3. Applying a headerlet to the science extensions

To apply/activate one of the other solutions, the `restore_from_headerlet()` function can be used.  This function applies the WCS contained in a HDRLET extension to all SCI extensions of the image.  Doing this requires knowing which solution should be applied, which can be obtained in multiple ways.

<div class="alert alert-block alert-warning">
NOTE: This is especially useful in cases where some of the exposures in a visit will have solutions that are aligned to Gaia, but others won't.  This is true for grism images in the same visit as direct images, or shallow/deep exposure combinations.
<div>

For instance, if the desired solution is `IDC_2731450pi-GSC240`, we can find the `EXTVER` of the corresponding HDRLET from the list of wcs names we generated before.

In [ ]:
# Gets the index of list element with value 'IDC_2731450pi-GSC240'
# The index in this list + 1 is the same as the EXTVER of the corresponding HDRLET
# We need to add 1 because lists are 0-indexed, while EXTVER's are 1 indexed
chosen_ext = new_wcsnames.index('IDC_2731450pi-GSC240') + 1

In [ ]:
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)

In this case we set `archive` keyword argument to `False`.  Setting `archive` to True will preserve the currently active WCS as a new HDRLET extension on the file.  Since in our case the current solution already has a HDRLET, we do not need to archive it.  This may be useful in some cases, such as when the image has been manually aligned/transformed, and keeping a record of that solution is desired.



We can check that the solution was applied:

In [ ]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

We can also apply the solution via the HDRNAME:

In [ ]:
hdrlet_hdrnames = headerlet.get_headerlet_kw_names(fits.open(filename), 'HDRNAME')
desired_hdrname = hdrlet_hdrnames[new_wcsnames.index('IDC_2731450pi-GSC240')]

In [ ]:
print(desired_hdrname)

In [ ]:
headerlet.restore_from_headerlet(filename, hdrname=desired_hdrname, archive=False, force=False)

We can also apply some logic to get the `hdrext` programatically.  For instance, if we only wanted the `IDC` (distortion calibrated) solution with the `GSC240` tag (indicating that the guide star positions had been updated), we can do the following:

In [ ]:
for i, wcsname in enumerate(new_wcsnames):
    if 'IDC' in wcsname and 'GSC240' in wcsname:
        chosen_ext = i + 1 # Add one due to 0 indexing of enumerate vs 1 indexing of EXTVER
        break

print('The desired extension is:', ('HDRLET',chosen_ext))
    

Finding the solution this way can be easier, as it doesn't require a full typing out of the IDCTAB name.  However, in the future, if new IDCTABs are created, there may be multiple solutions matching the criteria above, and more sophisticated logic will need to be applied.

## 4. Restoring the "old" solution

If the original solution is desired, it can be restored using the methods shown above, by replacing the WCSNAME simply with `IDC_2731450pi`, or whatever the name of the IDCTAB for that file is.  To get that name, the following procedure can be done:


<div class="alert alert-block alert-warning">
<b>NOTE:</b> Data taken after October 2017 may not have a solution of the form IDC_xxxxxxxxx, as the pointing information of the telescope was already calculated using GSC 2.4.0.  As such, the "old" solution may be of the same form as the a priori solution, i.e.: IDC_xxxxxxxxx-GSC240.
</div>

In [ ]:
idc_file = fits.getval(filename, 'IDCTAB')
idc_filename = idc_file.split('$')[-1]
idc_wcsname = 'IDC_' + idc_filename.replace('_idc.fits', '')

In [ ]:
print(idc_wcsname)

Restoring the solution is then done the same as above:

In [ ]:
chosen_ext = new_wcsnames.index(idc_wcsname) + 1

In [ ]:
headerlet.restore_from_headerlet(filename, hdrext=('HDRLET', chosen_ext), archive=False, force=False)

In [ ]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

Alternatively, `updatewcs` can be run on the image, with the `use_db` flag set to `False`:

In [ ]:
from stwcs import updatewcs
updatewcs.updatewcs(filename, use_db=False)

In [ ]:
current_wcs = fits.getval(filename, 'WCSNAME', ext=('SCI', 1))

print(current_wcs)

## 5. Inspecting alignment (optional)

While the solutions should improve the astrometry of the images, they may not have perfect absolute astrometry.  To check the quality of the absolute astrometry.  To do this, we can overplot the positions of the Gaia sources on the images.  If the absolute astrometry is correct, the Gaia positions will project on top of the sources in the images.  The WCS solutions that contain 'FIT-GAIA' in the WCSNAME should have the closest alignment, while the GSC-240 and baseline IDC solution will likely have small offsets.

In [3]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib notebook

We first need to get a listing of the Gaia sources:

In [4]:
from drizzlepac.alignimages import generate_astrometric_catalog
tbl = generate_astrometric_catalog('id7307xfq_flc.fits', output=True) 

The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       imagefindpars           mapreg              photeq       
     pixreplace           pixtopix            pixtosky        refimagefindpars  
     resetbits          runastrodriz          skytopix           tweakback      
      tweakreg           updatenpol
INFO: Created catalog 'ref_cat.ecsv' with 2132 sources
INFO: Wrote reference catalog refcatalog.cat.


In [5]:
tbl = Table.read('ref_cat.ecsv')

In [6]:
tbl

RA,DEC,mag,objID,GaiaID
float64,float64,float32,str19,str2
80.5229228711158,-67.9415992109552,-999.9,4658741577084029568,-1
80.5225590860532,-67.9432126755094,-999.9,4658741577083842688,-1
80.5214664507758,-67.9429492726399,-999.9,4658741581346201600,-1
80.5209998158025,-67.9429566282413,-999.9,4658741577084972416,-1
80.5213913893749,-67.9431990300295,-999.9,4658741581366226816,-1
80.5203786225385,-67.9426763962844,-999.9,4658741581346201216,-1
80.5228984977349,-67.9409753817395,-999.9,4658741577084029440,-1
80.5208596043234,-67.9436000476609,-999.9,4658741577062994688,-1
80.5212492055153,-67.9409686269975,-999.9,4658741581366510208,-1


We then need the WCS of the images, to project the Gaia positions onto the image coordinate frame.

In [ ]:
from astropy.wcs import WCS

In [ ]:
hdu = fits.open(filename)
w = WCS(hdu['SCI',1].header, hdu)
data = hdu['SCI',1].data

In [ ]:
x, y = w.all_world2pix(np.array([tbl['RA'], tbl['DEC']]).T, 0).T

In [ ]:
from ginga.util import zscale
z1, z2 = zscale.zscale(data)
fig = plt.figure(figsize=(10,5))
ax = plt.subplot(projection=w) # matplotlib has support for axes using WCS projections!

ax.imshow(data,origin='lower',vmin=z1,vmax=z2*30., norm=LogNorm())
ax.coords.grid(True, color='white', ls=':', alpha=.6)
ax.autoscale(False)
ax.scatter(x, y, edgecolors='r', facecolor=None, alpha=.8)
plt.show()

The original (only IDC tab) WCS will likely have an offset between the Gaia positions and the sources.  The GSC-240 solution offset is generally smaller, while any of the `GAIA-FIT` solutions should be well under a pixel.  Using the code in section 3, the offsets can be seen for each available WCS.

## 6. Changing the WCS of drizzled images

Because the drizzling process is directly affected by the WCS's of the input images, the WCS of the drizzled image cannot be changed as simply as shown above for FLC images.  To use an astrometric solution (other than the one  applied to the FLT/FLC at the time of drizzling), the images will have to be re-drizzled after activating the desired WCS in the FLT/FLC images.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> The images input to astrodrizzle should use the same WCS solution, or the drizzling process will produce poor results.
</div>

In [7]:
from drizzlepac import astrodrizzle

In order to get the same name for the drizzled file, we can simply run astrodrizzle on the association (asn) file we downloaded before.

In [8]:
astrodrizzle.AstroDrizzle('id7307030_asn.fits', mdriztab=True, build=True, clean=True, preserve=False)

Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.0.2 (2019-05-28 16:51:22 -0400) started at: 01:25:29.236 (03/12/2019)

==== Processing Step  Initialization  started at  01:25:29.238 (03/12/2019)
Reading in MDRIZTAB parameters for 4 files


##############################################################################
#                                                                            #
# ERROR:                                                                     #
# AstroDrizzle Version 3.0.2 encountered a problem!  Processing terminated   #
# at 01:25:29.505 (03/12/2019).                                              #
#                                                                            #
##############################################################################



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.0000 sec.
   ====================          ====================
                  Total          0.0000 sec.
Trailer file written to:  astrodrizzle.log


ValueError: Undefined variable `iref' in string `iref$2ck18260i_mdz.fits'

ValueError: Undefined variable `iref' in string `iref$2ck18260i_mdz.fits'